In [1]:
# need to fix the asyncio event loop
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Stan example
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...

In file included from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun.hpp:110,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/jovyan/.cache/httpstan/4.5.0/models/wfh3pgxe/model_wfh3pgxe.cpp:2:
/opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp: In function ‘Eigen::Matrix<typename stan::value_type<T>::type, EigMat::ColsAtCompileTime, EigMat::RowsAtCompileTime> stan::math::generalized_inverse(const EigMat&)’:
/opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp:34:9: warning: typedef ‘us

In file included from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/prob/multi_normal_cholesky_log.hpp:6,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/prob.hpp:196,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim.hpp:16,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:13,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/jovyan/.cache/httpstan/4.5.0/models/wfh3pgxe/model_wfh3pgxe.cpp:2:
/opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/prob/multi_normal_cholesky_lpdf.hpp: In function ‘stan::return_type_t<T_a, T_b, T_theta> stan::math::multi_normal_cholesky_lpdf(const T_y&, const T_loc&, const T_covar&)’:
/opt/conda/lib/pyt


Building: 17.5s, done.Messages from stanc:
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.11 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.13 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.13 seconds.
  Adjust your expectations accordingly!


In [3]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-39.401635,0.977998,0.382519,3.0,7.0,0.0,42.822742,9.208888,11.205474,0.823355,...,0.999706,1.764484,18.434972,6.022574,-14.637162,2.045767,11.657554,3.136816,20.411069,28.980762
1,-42.175409,0.868633,0.415691,4.0,15.0,0.0,45.215147,11.565739,2.514603,0.759946,...,-0.840000,-0.284040,13.476702,5.708840,9.893430,11.106762,12.471872,15.835802,9.453472,10.851491
2,-36.424391,0.973054,0.347654,3.0,15.0,0.0,40.540962,16.737697,11.680891,0.673622,...,-0.766648,-0.799979,24.606201,12.813749,10.450813,8.898678,6.597132,-0.834090,7.782564,7.393227
3,-39.380114,0.310979,0.385359,3.0,15.0,0.0,44.583715,15.779644,2.487793,0.330623,...,-1.221806,-0.140258,16.602165,15.068910,15.198818,12.816715,15.490446,15.310668,12.740044,15.430712
4,-42.962945,0.864946,0.382519,4.0,15.0,0.0,46.111289,6.075955,1.711160,-0.196013,...,-0.949492,-2.154642,5.740545,5.751012,9.700543,6.886554,3.871611,5.275496,4.451221,2.389017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-40.616958,0.881300,0.385359,3.0,7.0,0.0,41.808007,2.966995,3.646062,1.336780,...,1.730131,-0.356435,7.840977,-2.664392,7.528242,3.808392,0.566097,0.272874,9.275157,1.667411
3996,-41.035148,0.944745,0.382519,3.0,7.0,0.0,47.131483,7.622153,0.749102,0.243032,...,0.728132,0.374590,7.804208,8.574362,6.476871,8.113852,6.378374,7.757221,8.167598,7.902759
3997,-36.650918,0.994194,0.415691,3.0,7.0,0.0,41.985229,7.625578,7.630615,-0.718739,...,1.103411,0.454957,2.141159,4.767115,4.579025,10.599091,-1.651952,1.257540,16.045282,11.097181
